# Has Many Movie Lab

### Introduction
In this lab we will continue to look at the "Has-Many" relationships in our data. The database we will be using during this lab contains information about a selection of movies and related entities such as actors, directors and writers. A movie entity will have relationships with actor, director, and writer entities. The actors, directors and writers will also have relationships with themselves (i.e. a director will have worked with many actors). In problems below, we will use our knowledge of these relationships to build SQL queries.

Let's begin by connecting to the database and reviewing the schema of the tables.

In [ ]:
import sqlite3
conn = sqlite3.connect('films.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('SELECT name from sqlite_master where type= "table"')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(movies)')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(actors)')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(directors)')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(writers)')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(movie_actors)')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(movie_directors)')
cursor.fetchall()

In [ ]:
cursor.execute('PRAGMA table_info(movie_writers)')
cursor.fetchall()

### Questions

Let's start off with some basic one table queries:

* What is the title, runtime, and id of the movie with the longest runtime?

In [ ]:
cursor.execute("""
SELECT title,
       runtime,
       id
  FROM movies
 ORDER BY runtime DESC
 LIMIT 1;
""")
cursor.fetchall()

* Using your answer from the previous question, how many actors were credited for the movie with the longest runtime? Hint: Use the COUNT function with the movie ID

In [ ]:
cursor.execute("""
SELECT COUNT( * ) 
  FROM movie_actors
 WHERE movie_id = 480;

""")
cursor.fetchall()

* What was the shortest movie released in 2006?

In [ ]:
cursor.execute("""
SELECT title
  FROM movies
 WHERE year = 2006
 ORDER BY runtime
 LIMIT 1;
""")
cursor.fetchall()

### Has Many

* What are the names of the actors in Toy Story? (movie ID is 3648)

In [ ]:
cursor.execute("""
SELECT a.name
  FROM movie_actors AS ma
       JOIN
       actors AS a ON ma.actor_id = a.id
 WHERE ma.movie_id = 3648;
""")
cursor.fetchall()

* What is the name of the director of Toy Story?

In [ ]:
cursor.execute("""
SELECT d.name
  FROM movie_directors AS md
       JOIN
       directors AS d ON md.director_id = d.id
 WHERE md.movie_id = 3648;
""")
cursor.fetchall()

* What are the names of the writers of Toy Story?

In [ ]:
cursor.execute("""
SELECT w.name
  FROM movie_writers AS mw
       JOIN
       writers AS w ON mw.writer_id = w.id
 WHERE mw.movie_id = 3648;
""")
cursor.fetchall()

* What is the name and actor id of the actor with the most credits in the database?

In [ ]:
cursor.execute("""
SELECT a.name,
       a.id,
       COUNT( * ) 
  FROM movie_actors AS ma
       JOIN
       actors a ON ma.actor_id = a.id
 GROUP BY ma.actor_id
 ORDER BY COUNT( * ) DESC
 LIMIT 1;
""")
cursor.fetchall()

* What are the titles of the movies the actor from the previous question has been in, after the year 2005?

In [ ]:
cursor.execute("""
SELECT m.title
  FROM movie_actors AS ma
       JOIN
       movies AS m ON ma.movie_id = m.id
 WHERE ma.actor_id = 429 AND 
       m.year > 2005;
""")
cursor.fetchall()

* What are the titles of movies with more than two directors?

In [ ]:
cursor.execute("""
SELECT m.title
  FROM movie_directors AS md
       JOIN
       movies AS m ON md.movie_id = m.id
 GROUP BY m.id
HAVING COUNT( * ) > 2;
""")
# cursor.fetchall()

### Has Many Through

* What is the name of the writer in the database that has been credited the most times during the year 2018?

In [ ]:
cursor.execute("""
SELECT w.name,
       count( * ) 
  FROM movie_writers AS mw
       JOIN
       writers AS w ON mw.writer_id = w.id
       JOIN
       movies AS m ON mw.movie_id = m.id
 WHERE m.year = 2018
 GROUP BY w.id
 ORDER BY count( * ) DESC LIMIT 1;
""")
cursor.fetchall()

* What is the name of the actor or actress in the database that has been credited the most between 2010 and 2015 (inclusive)?

In [ ]:
cursor.execute("""
SELECT a.name,
       count( * ) 
  FROM movie_actors AS ma
       JOIN
       actors AS a ON ma.actor_id = a.id
       JOIN
       movies AS m ON ma.movie_id = m.id
 WHERE m.year >= 2010 AND m.year <= 2015
 GROUP BY a.id
 ORDER BY count( * ) DESC LIMIT 1;
""")
cursor.fetchall()

* What are the names of all actors who performed in more than 3 movies in 2010?

In [ ]:
cursor.execute("""
SELECT a.name
  FROM movie_actors AS ma
       JOIN
       movies AS m ON ma.movie_id = m.id
       JOIN
       actors AS a ON ma.actor_id = a.id
   WHERE m.year = 2010
   GROUP BY a.name
   HAVING COUNT(*) > 3;
""")
cursor.fetchall()

* What studio has Steven Spielberg worked with the most?

In [ ]:
cursor.execute("""
SELECT m.studio,
       COUNT( * ) 
  FROM movie_directors AS md
       JOIN
       movies AS m ON md.movie_id = m.id
       JOIN
       directors AS d ON md.director_id = d.id
 WHERE d.name = "Steven Spielberg"
 GROUP BY m.studio
 ORDER BY COUNT( * ) DESC
 LIMIT 1;
""")
cursor.fetchall()

* What years did Steven Spielberg direct 2 movies?

In [ ]:
cursor.execute("""
SELECT m.year,
       COUNT( * ) 
  FROM movie_directors AS md
       JOIN
       movies AS m ON md.movie_id = m.id
       JOIN
       directors AS d ON md.director_id = d.id
 WHERE d.name = "Steven Spielberg"
 GROUP BY m.year
 HAVING COUNT(*) = 2;
""")
cursor.fetchall()

* How many movies has each of the actors from Toy Story been in? (movie ID is 3648)

In [ ]:
cursor.execute("""
SELECT c.name,
       COUNT( * ) 
  FROM (
           SELECT actor_id
             FROM movie_actors
            WHERE movie_id = 3648
       )
       AS a
       JOIN
       movie_actors AS b ON a.actor_id = b.actor_id
       JOIN
       actors c ON a.actor_id = c.id
 GROUP BY a.actor_id;
""")
cursor.fetchall()

* What are the names of other movies the director of Toy Story directed? (movie ID is 3648)

In [ ]:
cursor.execute("""
SELECT title 
  FROM (
           SELECT director_id
             FROM movie_directors
            WHERE movie_id = 3648
       )
       AS a
       JOIN
       movie_directors AS b ON a.director_id = b.director_id
       JOIN
       movies c ON b.movie_id = c.id;
""")
cursor.fetchall()

* What are the names of all the directors Tom Hanks has worked with? (actor id 189)

In [ ]:
cursor.execute("""
SELECT DISTINCT c.name 
  FROM (
           SELECT movie_id
             FROM movie_actors
            WHERE actor_id = 189
       )
       AS a
       JOIN
       movie_directors AS b ON a.movie_id = b.movie_id
       JOIN
       directors c ON b.director_id = c.id;
""")
cursor.fetchall()

* What is the name of the director Tom Hanks has worked with the most?

In [ ]:
cursor.execute("""
SELECT c.name, COUNT(*) 
  FROM (
           SELECT movie_id
             FROM movie_actors
            WHERE actor_id = 189
       )
       AS a
       JOIN
       movie_directors AS b ON a.movie_id = b.movie_id
       JOIN
       directors c ON b.director_id = c.id
    GROUP BY c.name
    ORDER BY COUNT(*) DESC LIMIT 1;
""")
cursor.fetchall()

* What are the names of all the writers Tom Hanks has worked with?

In [ ]:
cursor.execute("""
SELECT DISTINCT c.name 
  FROM (
           SELECT movie_id
             FROM movie_actors
            WHERE actor_id = 189
       )
       AS a
       JOIN
       movie_writers AS b ON a.movie_id = b.movie_id
       JOIN
       writers c ON b.writer_id = c.id;
""")
cursor.fetchall()